In [1]:
import itertools

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

# VAE

In [2]:
class VAE(nn.Module):
    def __init__(self, in_channels=1, z_dim=32):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 4, kernel_size = 4, padding=1, stride=2), # conv1, input_channel -> 4
            nn.ReLU(), # relu
            nn.Conv2d(4, 8, kernel_size=4,padding=1, stride=2), # conv2, channel 4 -> 8
            nn.ReLU(), # relu
            nn.Conv2d(8, 16, kernel_size=4,padding=1, stride=2), # conv3, channel 8 -> 16
            nn.ReLU(), # relu
            nn.Conv2d(16, 32, kernel_size=4,padding=1, stride=2), # conv4, channel 16 -> 32
            nn.ReLU(), # relu
            nn.Flatten(), # flatten
        )
        
        # manually calculate the dimension after all convolutions
        dim_after_conv = 2
        hidden_dim = 32 * dim_after_conv * dim_after_conv
        
        self.readout_mu = nn.Linear(hidden_dim, z_dim)
        self.readout_sigma = nn.Linear(hidden_dim, z_dim)
        
        # You can use nn.ConvTranspose2d to decode
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, hidden_dim),
            nn.Unflatten(1, (32, dim_after_conv, dim_after_conv)),
            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1), # transpose-conv, channel 32 -> 16
            nn.ReLU(), # relu
            nn.ConvTranspose2d(16, 8, kernel_size=4, stride=2, padding=1), # transpose-conv, channel 16 -> 8
            nn.ReLU(), # relu
            nn.ConvTranspose2d(8, 4, kernel_size=4, stride=2, padding=1), # transpose-conv, channel 8 -> 4
            nn.ReLU(), # relu
            nn.ConvTranspose2d(4, 1, kernel_size=4, stride=2, padding=1), # transpose-conv, channel 4 -> input_channel, which is 1
            nn.Sigmoid(), # use a sigmoid activation to squeeze the outputs between 0 and 1
        )
    
    def reparameterize(self, mu, sigma):
        """
        Reparameterize, i.e. generate a z ~ N(\mu, \sigma)
        """
        # generate epsilon ~ N(0, I)
        # hint: use torch.randn or torch.randn_like
        epsilon = torch.rand_like(sigma)
        # z = \mu + \sigma * \epsilon
        z = mu + sigma + epsilon
        return z

    def encode(self, x):
        # call the encoder to map input to a hidden state vector
        h = self.encoder(x)
        # use the "readout" layer to get \mu and \sigma
        mu = self.readout_mu(h)
        sigma = self.readout_sigma(h)
        return mu, sigma

    def decode(self, z):
        # call the decoder to map z back to x
        return self.decoder(z)

    def forward(self, x):
        mu, sigma = self.encode(x)
        z = self.reparameterize(mu, sigma)
        x_recon = self.decode(z)
        return x_recon, mu, sigma

In [3]:
vae = VAE()
vae(torch.rand(10, 1, 32, 32))

(tensor([[[[0.5114, 0.5127, 0.5113,  ..., 0.5140, 0.5116, 0.5173],
           [0.5197, 0.5044, 0.5273,  ..., 0.5048, 0.5266, 0.5108],
           [0.5127, 0.5244, 0.5173,  ..., 0.5254, 0.5194, 0.5215],
           ...,
           [0.5182, 0.5051, 0.5268,  ..., 0.5060, 0.5257, 0.5109],
           [0.5133, 0.5246, 0.5175,  ..., 0.5249, 0.5179, 0.5234],
           [0.5194, 0.5157, 0.5207,  ..., 0.5151, 0.5206, 0.5174]]],
 
 
         [[[0.5114, 0.5127, 0.5113,  ..., 0.5139, 0.5114, 0.5174],
           [0.5198, 0.5045, 0.5272,  ..., 0.5047, 0.5272, 0.5108],
           [0.5128, 0.5245, 0.5174,  ..., 0.5255, 0.5194, 0.5213],
           ...,
           [0.5182, 0.5050, 0.5268,  ..., 0.5061, 0.5252, 0.5110],
           [0.5132, 0.5243, 0.5177,  ..., 0.5252, 0.5180, 0.5235],
           [0.5193, 0.5157, 0.5206,  ..., 0.5150, 0.5206, 0.5174]]],
 
 
         [[[0.5114, 0.5126, 0.5113,  ..., 0.5138, 0.5114, 0.5173],
           [0.5196, 0.5043, 0.5273,  ..., 0.5052, 0.5266, 0.5109],
           [0.5128

# GNN

In [4]:
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader as GraphDataLoader
from torch_geometric.utils import scatter

In [5]:
def load_qm9(path="./QM9"):
    def transform(data):
        edge_index = torch.tensor(
            list(itertools.permutations(range(data.x.shape[0]), 2)), 
            dtype=torch.long
        ).T
        edge_feature = 1 / torch.sqrt(
            torch.sum(
                (data.pos[edge_index[0]] - data.pos[edge_index[1]]) ** 2, 
                axis=1, keepdim=True
            )
        )
        data.edge_index = edge_index
        data.edge_attr = edge_feature
        data.y = data.y[:, [-7]]
        return data
    
    qm9 = QM9(path, transform=transform)
    return qm9

qm9 = load_qm9("../../Datasets/QM9")

In [6]:
class Layer(nn.Module):
    """
    Basic layer, a linear layer with a ReLU activation 
    """
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_dim, out_dim), # linear layer
            nn.ReLU() # relu
        )
    
    def forward(self, x):
        return self.layers(x)
    
    
class MessagePassingLayer(nn.Module):
    """
    A message passing layer that updates nodes/edge features
    """
    def __init__(self, node_hidden_dim, edge_hidden_dim):
        super().__init__()
        # figure out the input/output dimension
        self.edge_net = Layer(2*node_hidden_dim + edge_hidden_dim, edge_hidden_dim)
        # figure out the input/output dimension
        self.node_net = Layer(node_hidden_dim + edge_hidden_dim, node_hidden_dim)
    
    def forward(self, node_features, edge_features, edge_index):
        """
        Update node and edge features
        
        Parameters
        ----------
        node_features: torch.Tensor
            Node features from the previous layer
        edge_features: torch.Tensor
            Edge features from the previous layer
        edge_index: torch.Tensor
            A sparse matrix (n_edge, 2) in which each column denotes node indices forming an edge
        """
        # concatnate previous edge features with node features forming the edge
        # hint: use edge_features[edge_index[0(or 1)]] to get node features forming the edge
        concate_edge_features = torch.cat([
            node_features[edge_index[0]], # features of one node
            node_features[edge_index[1]], # features of the other node
            edge_features # previous edge features
        ], dim=1)
        
        # pass through the "edge_net" to map it back to the original dimension
        updated_edge_features = self.edge_net(concate_edge_features)
        
        
        # use scatter to aggrate the edge features to nodes
        aggr_edge_features = scatter(updated_edge_features, edge_index[0])
        # concatenate it with previous node features
        concate_node_features = torch.cat([aggr_edge_features, node_features], dim=1)
        # pass through the "node_net" to map it back to the original dimension
        updated_node_features = self.node_net(concate_node_features)
        
        return updated_node_features, updated_edge_features

        
class GraphNet(nn.Module):
    def __init__(self, node_input_dim, edge_input_dim, node_hidden_dim, edge_hidden_dim):
        super().__init__()
        # embed the input node features
        self.node_embed = Layer(node_input_dim, node_hidden_dim)
        # embed the input edge features
        self.edge_embed = Layer(edge_input_dim, edge_hidden_dim)
        # use a linear layer as readout to get the "atomic" energy contribution
        self.readout = nn.Linear(node_input_dim, 1)
        # message passing layer
        self.message_passing = MessagePassingLayer(node_hidden_dim, edge_hidden_dim) 
    
    def forward(self, node_features, edge_features, edge_index, batch):
        """
        Update node and edge features
        
        Parameters
        ----------
        node_features: torch.Tensor
            Node features from the previous layer
        edge_features: torch.Tensor
            Edge features from the previous layer
        edge_index: torch.Tensor
            A sparse matrix (n_edges, 2) in which each column denotes node indices forming an edge
        batch: torch.Tensor
            A 1-D tensor (n_nodes,) that tells you each node belongs to which graph
        """
        node_hidden = self.node_embed(node_features) # call the node embedding
        edge_hidden = self.edge_embed(edge_features) # call the edge embedding
        updated_node_hidden, updated_edge_hidden = self.message_passing(
            node_hidden,
            edge_hidden,
            edge_index) # call the message passing layer
        readout = self.readout(updated_node_hidden) # use the readout layer to output "atomic" contributions
        out = scatter(readout, batch) # use the scatter function to aggregate atomic readouts
        return out

In [7]:
node_input_dim = 11
edge_input_dim = 1
node_hidden_dim = 64
edge_hidden_dim = 64

net = GraphNet(node_input_dim, edge_input_dim, node_hidden_dim, edge_hidden_dim)

In [ ]:
batch_data = next(iter(GraphDataLoader(qm9[:10], batch_size=2)))
batch_pred = net(
    batch_data.x, batch_data.edge_attr, 
    batch_data.edge_index, batch_data.batch
)
batch_pred